# Module 2: Advanced Blockchain Features

## Overview
This notebook explores advanced blockchain features, focusing on transactions, the UTXO model, wallets, and smart contracts. By the end of this module, you will understand:

1. Transaction structures and validation
2. The Unspent Transaction Output (UTXO) model
3. Wallet creation and management
4. Smart contract fundamentals

## Table of Contents
1. [Transactions](#transactions)
2. [UTXO Model](#utxo)
3. [Wallets](#wallets)
4. [Smart Contracts](#smart-contracts)
5. [Implementation Examples](#implementation)
6. [Summary](#summary)

<a id='transactions'></a>
## 1. Transactions

### What is a Transaction?
A transaction is a transfer of value between participants in the blockchain network. It represents the atomic unit of change in the blockchain state.

### Transaction Structure
A transaction typically consists of:
1. **Inputs**: References to previous transaction outputs that fund this transaction
2. **Outputs**: New ownership claims on cryptocurrency
3. **Metadata**: Additional information like fees, timestamps, etc.

### Transaction Components
```go
// Transaction represents a transaction in the blockchain
type Transaction struct {
    ID   []byte
    Vin  []TXInput
    Vout []TXOutput
}

// TXInput represents a transaction input
type TXInput struct {
    Txid      []byte  // ID of the transaction this input references
    Vout      int     // Index of the output in the referenced transaction
    Signature []byte  // Cryptographic signature
    PubKey    []byte  // Public key of the sender
}

// TXOutput represents a transaction output
type TXOutput struct {
    Value      int    // Amount of cryptocurrency
    PubKeyHash []byte // Hash of the recipient's public key
}
```

### Coinbase Transactions
The first transaction in a block is special and is called a coinbase transaction. It creates new cryptocurrency and rewards the miner.

```go
// NewCoinbaseTX creates a new coinbase transaction
func NewCoinbaseTX(to, data string) *Transaction {
    if data == "" {
        data = fmt.Sprintf("Reward to \"%s\"", to)
    }

    txin := TXInput{[]byte{}, -1, nil, []byte(data)}
    txout := NewTXOutput(10, to)
    tx := Transaction{nil, []TXInput{txin}, []TXOutput{*txout}}
    tx.ID = tx.Hash()

    return &tx
}
```

<a id='utxo'></a>
## 2. UTXO Model

### What is UTXO?
UTXO stands for Unspent Transaction Output. It's a model for tracking ownership of cryptocurrency where:
- Each transaction consumes previous UTXOs as inputs
- Each transaction creates new UTXOs as outputs
- Only unspent outputs can be used as inputs for new transactions

### How UTXO Works
1. When Alice wants to send cryptocurrency to Bob, she must find UTXOs that sum to at least the amount she wants to send
2. These UTXOs become inputs to her new transaction
3. The transaction creates new outputs:
   - One to Bob (the payment)
   - One back to Alice (change, if any)
4. The original UTXOs are now considered "spent" and cannot be used again

### UTXO Set Management
To efficiently track UTXOs, we maintain a UTXO set:

```go
// FindSpendableOutputs finds and returns unspent outputs to reference in inputs
func (bc *Blockchain) FindSpendableOutputs(pubkeyHash []byte, amount int) (int, map[string][]int) {
    unspentOutputs := make(map[string][]int)
    unspentTXs := bc.FindUnspentTransactions(pubkeyHash)
    accumulated := 0

Work:
    for _, tx := range unspentTXs {
        txID := hex.EncodeToString(tx.ID)

        for outIdx, out := range tx.Vout {
            if out.IsLockedWithKey(pubkeyHash) && accumulated < amount {
                accumulated += out.Value
                unspentOutputs[txID] = append(unspentOutputs[txID], outIdx)

                if accumulated >= amount {
                    break Work
                }
            }
        }
    }

    return accumulated, unspentOutputs
}
```

<a id='wallets'></a>
## 3. Wallets

### What is a Wallet?
A wallet is a software application that stores private keys and provides an interface for users to interact with the blockchain.

### Wallet Components
1. **Private Key**: Secret key used for signing transactions
2. **Public Key**: Derived from the private key, used to receive funds
3. **Address**: A human-readable representation of the public key hash

### Wallet Structure
```go
// Wallet represents a wallet
type Wallet struct {
    PrivateKey []byte
    PublicKey  []byte
}

// Wallets represents a collection of wallets
type Wallets struct {
    Wallets map[string]*Wallet
}
```

### Wallet Creation
```go
// NewWallet creates and returns a Wallet
func NewWallet() *Wallet {
    private, public := newKeyPair()
    wallet := Wallet{private, public}

    return &wallet
}

// GetAddress returns wallet address
func (w Wallet) GetAddress() []byte {
    pubKeyHash := HashPubKey(w.PublicKey)
    versionedPayload := append([]byte{version}, pubKeyHash...)
    checksum := checksum(versionedPayload)

    fullPayload := append(versionedPayload, checksum...)
    address := Base58Encode(fullPayload)

    return address
}
```

<a id='smart-contracts'></a>
## 4. Smart Contracts

### What are Smart Contracts?
Smart contracts are self-executing contracts with the terms of the agreement directly written into code. They automatically execute when predetermined conditions are met.

### Key Characteristics
1. **Autonomous**: Execute automatically when conditions are met
2. **Decentralized**: Run on the blockchain network
3. **Transparent**: Code and execution are visible to all participants
4. **Irreversible**: Once deployed, they cannot be altered

### Smart Contract Structure
```go
// SmartContract represents a smart contract
type SmartContract struct {
    ID          []byte
    Code        []byte
    State       map[string]interface{}
    Balance     int
    Owner       []byte
    Permissions map[string][]string
}

// SmartContractVM represents a virtual machine for executing smart contracts
type SmartContractVM struct {
    StateDB map[string]map[string]interface{}
    Gas     int
}
```

### Smart Contract Execution
```go
// Execute runs the smart contract code
func (sc *SmartContract) Execute(vm *SmartContractVM, function string, args ...interface{}) (interface{}, error) {
    // Check if caller has permission to execute this function
    // Deduct gas fees
    // Execute the function
    // Update state
    
    switch function {
    case "transfer":
        return sc.transfer(args...)
    case "balanceOf":
        return sc.balanceOf(args...)
    default:
        return nil, fmt.Errorf("unknown function: %s", function)
    }
}
```

<a id='implementation'></a>
## 5. Implementation Examples

### Creating a Transaction
```go
// NewTransaction creates a new transaction
func NewTransaction(from, to string, amount int, bc *Blockchain) *Transaction {
    var inputs []TXInput
    var outputs []TXOutput

    // Get wallet for the sender
    wallets, err := NewWallets()
    if err != nil {
        fmt.Printf("Error: %s\n", err)
        return nil
    }
    wallet := wallets.GetWallet(from)
    pubKeyHash := HashPubKey(wallet.PublicKey)

    // Find spendable outputs
    acc, validOutputs := bc.FindSpendableOutputs(pubKeyHash, amount)

    if acc < amount {
        fmt.Printf("Error: Not enough funds\n")
        return nil
    }

    // Build a list of inputs
    for txid, outs := range validOutputs {
        txID, err := hex.DecodeString(txid)
        if err != nil {
            fmt.Printf("Error: %s\n", err)
            return nil
        }

        for _, out := range outs {
            input := TXInput{txID, out, nil, wallet.PublicKey}
            inputs = append(inputs, input)
        }
    }

    // Build a list of outputs
    outputs = append(outputs, *NewTXOutput(amount, to))
    if acc > amount {
        outputs = append(outputs, *NewTXOutput(acc-amount, from)) // a change
    }

    tx := Transaction{nil, inputs, outputs}
    tx.ID = tx.Hash()
    bc.SignTransaction(&tx, wallet.PrivateKey)

    return &tx
}
```

<a id='summary'></a>
## 6. Summary

In this module, we've covered advanced blockchain features:

1. **Transactions**: The fundamental unit of value transfer in blockchain systems
2. **UTXO Model**: A method for tracking cryptocurrency ownership
3. **Wallets**: Tools for managing private keys and interacting with the blockchain
4. **Smart Contracts**: Self-executing contracts with terms written in code

### Next Steps
In Module 3, we'll explore networking and consensus mechanisms:
- Peer-to-peer network implementation
- Node discovery protocols
- Consensus algorithms

### Key Takeaways
- Transactions are the core mechanism for transferring value
- The UTXO model provides a secure way to track ownership
- Wallets are essential for user interaction with blockchain systems
- Smart contracts enable programmable blockchain applications

### Practice Exercises
1. Implement a simple transaction creation function
2. Create a wallet and generate an address
3. Build a UTXO set manager
4. Write a basic smart contract with a transfer function